# Final Project: Online Retail Data Anaysis
Description: This project is to analyse the Transaction Level Retail Data for the customers and recommend additional products for the customers
    
    
Author: Rajesh Kumar
Date: 12/16/2017

# Basic Data Loading


In [67]:
# NumPy for numerical computing
import numpy as np #linear algebra
# Pandas for DataFrames
import pandas as pd #data processing
# Matplotlib for visualization
import matplotlib.pyplot as plt
# Seaborn for easier visualization
import seaborn as sns

#sklearn to do item-item clusterring 
from sklearn import metrics
from sklearn.cluster import KMeans

In [68]:
data = pd.read_csv('onlineretaildatasmall.csv')

C:\Users\User\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      95604 non-null object
StockCode      95604 non-null object
Description    95308 non-null object
Quantity       95604 non-null float64
InvoiceDate    95604 non-null object
UnitPrice      95604 non-null float64
CustomerID     61961 non-null float64
Country        95604 non-null object
dtypes: float64(3), object(5)
memory usage: 33.1+ MB


In [70]:
data.index

RangeIndex(start=0, stop=541909, step=1)

In [71]:
#coorelation map
f,ax = plt.subplots(figsize=(18,18))
sns.heatmap(data.corr(), annot=True, linewidths=.5, fmt='.1f', ax=ax)


In [72]:
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2.0,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6.0,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6.0,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6.0,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [73]:
#explore number of unique countries
len(data.Country.unique())

30

In [74]:
#create a dataframe for UK
datauk = data.loc[data.Country == 'United Kingdom']

In [75]:
datauk = datauk.sort_values(['InvoiceNo']).reset_index(drop=True)
datauk.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [76]:
datauk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88348 entries, 0 to 88347
Data columns (total 8 columns):
InvoiceNo      88348 non-null object
StockCode      88348 non-null object
Description    88052 non-null object
Quantity       88348 non-null float64
InvoiceDate    88348 non-null object
UnitPrice      88348 non-null float64
CustomerID     54859 non-null float64
Country        88348 non-null object
dtypes: float64(3), object(5)
memory usage: 5.4+ MB


In [79]:
invoice_cutoff = '542896'   # approx 80% of data for training
train=datauk.loc[datauk.InvoiceNo<=invoice_cutoff]
test=datauk.loc[datauk.InvoiceNo>invoice_cutoff]
users_train=set(list(train.CustomerID))
items_train=set(list(train.StockCode))
test=test.loc[test.CustomerID.map(lambda x: x in users_train)]
test=test.loc[test.StockCode.map(lambda x: x in items_train)]
print(train.shape)
print(test.shape)

(71814, 8)
(7218, 8)


# Train and Test split

In [80]:
print(len(users_train))
print(len(items_train))

30021
3029


In [81]:
from cmfrec import CMF
import numpy as np

# Number of latent factors
k=40

# Regularization parameter
reg=1e-3

# Fitting the model
rec=CMF(k=k, reg_param=reg)
rec.fit(train, random_seed=12345)

# Making predictions
test['Predicted']=test.apply(lambda x: rec.predict(x['CustomerID'],x['StockCode']),axis=1)


ImportError: No module named cmfrec

# Cluster Analysis

In [78]:
##
# Do K-Means clustering to find recommendation
#
##